# PUMP IT UP


* [Git Hub Repo](https://github.com/msampathkumar/datadriven_pumpit)
* [Git Hub Report](https://github.com/msampathkumar/datadriven_pumpit/blob/master/capstone_proposal.mdown)
* [Features Details](https://www.drivendata.org/competitions/7/page/25/)


TODO:

1. Variance Threshold Dict
2. know variance threshold for removed columns in 


# Import Libraries

In [1]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

np.random.seed(69572)

%matplotlib inline

%load_ext writeandexecute

# plt.figure(figsize=(120,10))

small = (4,3)
mid = (10, 8)
large = (12, 8)

'writeandexecute' magic loaded.


# Custom Hacks

### MarkUP Fns

In [ ]:
from __future__ import absolute_import
from IPython.core.getipython import get_ipython
from IPython.core.magic import (Magics, magics_class,  cell_magic)
import sys
from StringIO import StringIO
from markdown import markdown
from IPython.core.display import HTML
 
@magics_class
class MarkdownMagics(Magics):
 
    @cell_magic
    def asmarkdown(self, line, cell):
        buffer = StringIO()
        stdout = sys.stdout
        sys.stdout = buffer
        try:
            exec(cell, locals(), self.shell.user_ns)
        except:
            sys.stdout = stdout
            raise
        sys.stdout = stdout
        return HTML("<p>{}</p>".format(markdown(buffer.getvalue(), extensions=['markdown.extensions.extra'])))
        return buffer.getvalue() + 'test'
 
get_ipython().register_magics(MarkdownMagics)

### Value Counts

In [ ]:
def raw_markup_value_counts(dataframe, max_print_value_counts=30, show_plots=False, figsize=(9, 3)):
    '''
    prints value counts of each feature in data frame
    '''
    if not figsize:
        figsize=(9, 3)
    mydf = pd.DataFrame.copy(dataframe)
    i = 0
    raw_markup_data = []
    pp = raw_markup_data.append
    pp('''|Col ID|Col Name|UniqCount|Col Values|UniqValCount|''')
    pp('''|------|--------|---------|----------|------------|''')
    for col in mydf.dtypes.index:
        i += 1
        sam = mydf[col]
        sam_value_counts = sam.value_counts()
        tmp = len(sam_value_counts)
        sam_value_counts_len = len(sam_value_counts)
        if 1 < sam_value_counts_len < max_print_value_counts:
            flag = True
            for key, val in dict(sam_value_counts).iteritems():
                if flag:
                    pp('|%i|%s|%i|%s|%s|' % (
                            i, col, sam_value_counts_len, key, val))
                    flag = False
                else:
                    pp('||-|-|%s|%s|' % (key, val))
            if show_plots:
                plt.figure(i)
                ax = sam_value_counts.plot(kind='barh', figsize=figsize)
                _ = plt.title(col.upper())
                _ = plt.xlabel('counts')
        else:
            pp('|%i|%s|%i|||' % (i, col, sam_value_counts_len))
    return raw_markup_data

### Confusion Matrix

In [ ]:
from __future__ import division
import itertools
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')    

    
def confusion_maxtrix_stuff(y_test, y_pred, class_names):
    '''
    Example
    >>> confusion_maxtrix_stuff(y_test,
                                y_pred,
                                class_names=RAW_y.status_group.value_counts().keys()
                                ):
    '''
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    plt.figure(figsize=(8,8))
    plot_confusion_matrix(cnf_matrix, classes=class_names,
                          title='Confusion matrix, without normalization')
    # Plot normalized confusion matrix
    plt.figure(figsize=(8,8))
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized confusion matrix')
    plt.show()

## Import CSV Data

In [ ]:
RAW_X = pd.read_csv('traning_set_values.csv', index_col='id')
RAW_y = pd.read_csv('training_set_labels.csv', index_col='id')
RAW_TEST_X = pd.read_csv('test_set_values.csv', index_col='id')

In [ ]:
RAW_X.head().T

** Observations: **

* Most of the data seems categorical
* Following pairs looks closesly related
    * quantity & quantity_group
    * quality_group & water_quality
    * extraction_type, extraction_type_class & extraction_type_group
* Other
    * recorded_by, seems to hold only a single value
    * population & amount_tsh, values are for some given as zero

Lets check those group with 

In [ ]:
# %%asmarkdown
# show me the graphs
tmp = raw_markup_value_counts(dataframe=RAW_X, max_print_value_counts=10, show_plots=True, figsize=(9, 2))

## Pre Processing

** Num/Bool Tranformations **

* date_recorded --> Int
* longitude --> Float(less precision)
* latitude --> Float(less precision)
* public_meeting --> Bool
* permit --> Bool


Precision Description of Longititude and Latitude is available here at below link.
* http://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude

In [ ]:
from datetime import datetime
strptime = datetime.strptime

DATE_FORMAT = "%Y-%m-%d"
REFERENCE_DATE_POINT = strptime('2000-01-01', DATE_FORMAT)

# Reducing geo location precision to 11 meters
LONG_LAT_PRECISION = 0.001

def sam_datetime_to_number(x):
    return (strptime(str(x), DATE_FORMAT) - REFERENCE_DATE_POINT).days


# Transforming Date to Int.
if RAW_X.date_recorded.dtype == 'O':
    RAW_X.date_recorded = RAW_X.date_recorded.map(sam_datetime_to_number)
    RAW_TEST_X.date_recorded = RAW_TEST_X.date_recorded.map(sam_datetime_to_number)


# Filling Missing/OUTLIAR Values
_ = np.mean(RAW_X[u'latitude'][RAW_X.latitude < -1.0].values)
if not RAW_X.loc[RAW_X.latitude >= -1.0, u'latitude'].empty:
    RAW_X.loc[RAW_X.latitude >= -1.0, u'latitude'] = _
    RAW_TEST_X.loc[RAW_TEST_X.latitude >= -1.0, u'latitude'] = _


# Filling Missing/OUTLIAR Values
_ = np.mean(RAW_X[u'longitude'][RAW_X[u'longitude'] > 1.0].values)
if not RAW_X.loc[RAW_X[u'longitude'] <= 1.0, u'longitude'].empty:
    RAW_X.loc[RAW_X[u'longitude'] <= 1.0, u'longitude'] = _
    RAW_TEST_X.loc[RAW_TEST_X[u'longitude'] <= 1.0, u'longitude'] = _


# Reducing Precision of Lat.
if RAW_X.longitude.mean() < 50:
    RAW_X.longitude = RAW_X.longitude // LONG_LAT_PRECISION
    RAW_X.latitude = RAW_X.latitude // LONG_LAT_PRECISION
    RAW_TEST_X.longitude = RAW_TEST_X.longitude // LONG_LAT_PRECISION
    RAW_TEST_X.latitude = RAW_TEST_X.latitude // LONG_LAT_PRECISION


# Filling Missing/OUTLIAR Values
if RAW_X.public_meeting.dtype != 'bool':
    RAW_X.public_meeting = RAW_X.public_meeting == True
    RAW_TEST_X.public_meeting = RAW_TEST_X.public_meeting == True

if RAW_X.permit.dtype != 'bool':
    RAW_X.permit = RAW_X.permit == True
    RAW_TEST_X.permit = RAW_TEST_X.permit == True


# checking
if list(RAW_TEST_X.dtypes[RAW_TEST_X.dtypes != RAW_X.dtypes]):
    raise Exception('RAW_X.dtypes and RAW_TEST_X.dtypes are not in Sync')
else:
    print 'All in Good Shape'

** Text Data Tranformations **

We are going to basic clean action like, lower and upper case issue. Clearning of non ascii values.

In [ ]:
def text_transformation(name):
    if name:
        name = name.lower().strip()
        name = ''.join([i if 96 < ord(i) < 128 else ' ' for i in name])
        if 'and' in name:
            name = name.replace('and', ' ')
        if '/' in name:
            name = name.replace('/', ' ')
        while '  ' in name:
            name = name.replace('  ', ' ')
        return name.strip()
    return

In [ ]:
%%asmarkdown

print '''
|Column|Prev.|Current|
|------|-----|-------|'''
for col in RAW_X.dtypes[RAW_X.dtypes == object].index:
    aa = len(RAW_X[col].unique())
    RAW_X[col] = RAW_X[col].fillna('').apply(text_transformation)
    RAW_TEST_X[col] = RAW_TEST_X[col].fillna('').apply(text_transformation)
    bb = len(RAW_X[col].unique())
    if aa != bb:
        print '|%s|%i|%i|' % (col, aa, bb)

## Label Encoder

Label Encoder with DefaultDict for quick data transformation
http://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

In [ ]:
from collections import defaultdict
from sklearn import preprocessing

In [ ]:
d = defaultdict(preprocessing.LabelEncoder)

# Labels Fit
sam = pd.concat([RAW_X, RAW_TEST_X]).apply(lambda x: d[x.name].fit(x))

# Labels Transform - Training Data
X = RAW_X.apply(lambda x: d[x.name].transform(x))
TEST_X = RAW_TEST_X.apply(lambda x: d[x.name].transform(x))

le = preprocessing.LabelEncoder().fit(RAW_y[u'status_group'])
y = le.transform(RAW_y[u'status_group'])

In [ ]:
X.head(5)

In [ ]:
y[:5]

## Pickle Save

In [ ]:
pickle.dump(X, open('X.pkl', 'w'))
pickle.dump(TEST_X, open('TEST_X.pkl', 'w'))
pickle.dump(y, open('y.pkl', 'w'))
pickle.dump(d, open('d.pkl', 'w'))
pickle.dump(le, open('le.pkl', 'w'))

## Pickle Load

In [2]:
X = pickle.load(open('X.pkl'))
y = pickle.load(open('y.pkl'))

In [3]:
# Load this when you are about to do text transformation and submission

TEST_X = pickle.load(open('TEST_X.pkl'))
d = pickle.load(open('d.pkl'))
le = pickle.load(open('le.pkl'))

In [4]:
X.shape, y.shape

((59400, 39), (59400,))

In [5]:
y[:5]

array([0, 0, 0, 2, 0])

# Feature Selection


### Variance Threshold

To remove all features that are either one or zero (on or off) in more than 80% of the samples.

http://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance

http://stackoverflow.com/questions/29298973/removing-features-with-low-variance-scikit-learn/34850639#34850639

In [6]:
from sklearn.feature_selection import VarianceThreshold

def get_low_variance_columns(dframe=None, columns=[],
                             skip_columns=[], threshold=0.0,
                             autoremove=False):
    """
    Wrapper for sklearn VarianceThreshold for use on pandas dataframes.
    """
    print("Finding low-variance features.")
    removed_features = []
    ranking_variance_thresholds = {}
    try:
        # get list of all the original df columns
        all_columns = dframe.columns

        # remove `skip_columns`
        remaining_columns = all_columns.drop(skip_columns)

        # get length of new index
        max_index = len(remaining_columns) - 1

        # get indices for `skip_columns`
        skipped_idx = [all_columns.get_loc(col)
                       for col
                       in skip_columns]

        # adjust insert location by the number of columns removed
        # (for non-zero insertion locations) to keep relative
        # locations intact
        for idx, item in enumerate(skipped_idx):
            if item > max_index:
                diff = item - max_index
                skipped_idx[idx] -= diff
            if item == max_index:
                diff = item - len(skip_columns)
                skipped_idx[idx] -= diff
            if idx == 0:
                skipped_idx[idx] = item

        # get values of `skip_columns`
        skipped_values = dframe.iloc[:, skipped_idx].values

        # get dataframe values
        X = dframe.loc[:, remaining_columns].values

        # instantiate VarianceThreshold object
        vt = VarianceThreshold(threshold=threshold)

        # fit vt to data
        vt.fit(X)

        # threshold ranking
        ranking_variance_thresholds = dict(zip(remaining_columns, vt.variances_))

        # get the indices of the features that are being kept
        feature_indices = vt.get_support(indices=True)

        # remove low-variance columns from index
        feature_names = [remaining_columns[idx]
                         for idx, _
                         in enumerate(remaining_columns)
                         if idx
                         in feature_indices]

        # get the columns to be removed
        removed_features = list(np.setdiff1d(remaining_columns,
                                             feature_names))
        print("Found {0} low-variance columns."
              .format(len(removed_features)))

        # remove the columns
        if autoremove:
            print("Removing low-variance features.")
            # remove the low-variance columns
            X_removed = vt.transform(X)

            print("Reassembling the dataframe (with low-variance "
                  "features removed).")
            # re-assemble the dataframe
            dframe = pd.DataFrame(data=X_removed,
                                  columns=feature_names)

            # add back the `skip_columns`
            for idx, index in enumerate(skipped_idx):
                dframe.insert(loc=index,
                              column=skip_columns[idx],
                              value=skipped_values[:, idx])
            print("Succesfully removed low-variance columns.")

        # do not remove columns
        else:
            print("No changes have been made to the dataframe.")

    except Exception as e:
        print(e)
        print("Could not remove low-variance features. Something "
              "went wrong.")
        return dframe, [], {}

    return dframe, removed_features, ranking_variance_thresholds

In [7]:
X, removed_features, ranking_variance_thresholds = get_low_variance_columns(dframe=X,
                                                                            threshold=(0.85 * (1 - 0.85)),
                                                                            autoremove=True)

print '\nLow Variance Columns', removed_features
print 'Shape of X is', X.shape

Finding low-variance features.
Found 2 low-variance columns.
Removing low-variance features.
Reassembling the dataframe (with low-variance features removed).
Succesfully removed low-variance columns.

Low Variance Columns ['public_meeting', 'recorded_by']
Shape of X is (59400, 37)


In [8]:
TEST_X.drop(removed_features, axis=1, inplace=True)

In [9]:
print 'Shape of X is', X.shape
print 'Shape of TEST_X is', TEST_X.shape

Shape of X is (59400, 37)
Shape of TEST_X is (14850, 37)


### Select K Best

* For regression: f_regression, mutual_info_regression
* For classification: chi2, f_classif, mutual_info_classif


Random Forest Classifier score: RandomForestClassifier(n_estimators=150, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1)
* chi2 0.81225589225589223
*  f_classic 0.81138047138047142
* mutual_info_classif 0.81037037037037041

In [11]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

In [12]:
K_BEST_COLS = 25

fit = SelectKBest(score_func=chi2, k=K_BEST_COLS).fit(X, y)
cols_names = X.columns

np.set_printoptions(precision=2)

ranking_selectkbest = dict(zip(cols_names, fit.scores_))
selected_cols =  [_ for _ in cols_names[:K_BEST_COLS]]

features = pd.DataFrame(fit.transform(X))
features.columns = selected_cols

In [13]:
# % pprint
ranking_selectkbest

{'amount_tsh': 74881.491646077397,
 'basin': 266.29188142394912,
 'construction_year': 24316.883191063331,
 'date_recorded': 9583.0078737288022,
 'district_code': 644.5891174545709,
 'extraction_type': 2638.1965790559675,
 'extraction_type_class': 4962.445268561687,
 'extraction_type_group': 3427.7617911925518,
 'funder': 71952.584999370752,
 'gps_height': 512717.9078653045,
 'installer': 24259.764104192629,
 'latitude': 52365.996981064389,
 'lga': 9145.8259153344916,
 'longitude': 485645.5061894512,
 'management': 176.68024996472877,
 'management_group': 30.233212840532243,
 'num_private': 210.93553962837058,
 'payment': 866.20357165258588,
 'payment_type': 462.55917116540058,
 'permit': 21.535401607392107,
 'population': 5162.4773551716098,
 'quality_group': 422.67270817801528,
 'quantity': 672.8652286072313,
 'quantity_group': 672.8652286072313,
 'region': 1805.634614408988,
 'region_code': 1704.2139133512608,
 'scheme_management': 439.39640906001443,
 'scheme_name': 77768.875220045

In [13]:
print X.shape, features.shape, len(y)

(59400, 37) (59400, 30) 59400


In [14]:
print 'Removed Columns:\n\t', '\n\t'.join([ _ for _ in X.columns if _ not in features.columns ])

print '\nSelected Columns:\n\t', '\n\t'.join(features.columns)

Removed Columns:
	management_group
	payment
	payment_type
	water_quality
	quality_group
	quantity
	quantity_group
	source
	source_type
	source_class
	waterpoint_type
	waterpoint_type_group

Selected Columns:
	amount_tsh
	date_recorded
	funder
	gps_height
	installer
	longitude
	latitude
	wpt_name
	num_private
	basin
	subvillage
	region
	region_code
	district_code
	lga
	ward
	population
	scheme_management
	scheme_name
	permit
	construction_year
	extraction_type
	extraction_type_group
	extraction_type_class
	management


In [15]:
print X.shape, features.shape

(59400, 37) (59400, 25)


In [16]:
X = pd.DataFrame(fit.transform(X))
X.columns = selected_cols

print

# TEST_X = pd.DataFrame(fit.transform(TEST_X))
# TEST_X.columns = selected_cols

In [17]:
TEST_X = pd.DataFrame(fit.transform(TEST_X))

print TEST_X.shape, X.shape

(14850, 25) (59400, 25)


In [18]:
pickle.dump(X, open('processed_X.pkl', 'w'))
pickle.dump(TEST_X, open('processed_TEST_X.pkl', 'w'))
pickle.dump(y, open('processed_y.pkl', 'w'))

### PCA

In [ ]:
X = pickle.load(open('processed_X.pkl'))
TEST_X = pickle.load(open('processed_TEST_X.pkl'))
y = pickle.load(open('processed_y.pkl'))

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# feature extraction
pca = PCA(n_components=7)
fit = pca.fit(X)

In [ ]:
plt.figure(figsize=(12, 3))

_ = plt.scatter (range(len(fit.explained_variance_ratio_)), fit.explained_variance_ratio_.cumsum())

_ = plt.xlabel('cumsum of explained variance')

print fit.explained_variance_ratio_.cumsum()

In [ ]:
X = pca.transform(X)
TEST_X = pca.transform(TEST_X)

In [ ]:
pickle.dump(X, open('processed_X.pkl', 'w'))
pickle.dump(TEST_X, open('processed_TEST_X.pkl', 'w'))
pickle.dump(y, open('processed_y.pkl', 'w'))

## Unsupervised Learning

* Unsupervised Learning Exploration(Gaussian Process, Neural Nets)

In [ ]:
X = pickle.load(open('processed_X.pkl'))
TEST_X = pickle.load(open('processed_TEST_X.pkl'))
y = pickle.load(open('processed_y.pkl'))

In [19]:
print X.shape, y.shape, TEST_X.shape

(59400, 25) (59400,) (14850, 25)


### Gaussian Process

In [20]:
from sklearn import mixture
import itertools
from scipy import linalg
import matplotlib as mpl

try:
    import renders as rs
except:
    pass
import seaborn as sb
from IPython.display import display

In [21]:
from sklearn.mixture import GaussianMixture as GMM
from sklearn.metrics import silhouette_score

# For future analysis
GMM_Centers = []

In [22]:
i = 2
clusterer = GMM(n_components=i, random_state=42)
clusterer.fit(X)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=2, n_init=1, precisions_init=None,
        random_state=42, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [23]:
preds = clusterer.predict(X)
GMM_Centers.append(clusterer.means_)
print clusterer.means_

[[  8.01e+00   1.76e+02   9.86e+02   4.87e+02   8.30e+02   4.87e+03
    5.54e+03   2.52e+04   1.06e+04   1.24e+01   1.31e+01   3.81e+00
    5.93e+01   1.03e+03   1.44e+02   1.68e+02   2.07e+01   8.08e+00
    5.85e+00   1.39e+00   2.17e+00   1.35e+00   1.35e+00   3.77e+00
    2.84e+00]
 [  1.01e+01   1.96e+02   8.25e+02   8.37e+02   6.59e+02   5.51e+03
    5.25e+03   2.47e+04   1.06e+04   8.68e+00   1.01e+01   3.89e+00
    6.04e+01   1.07e+03   1.41e+02   9.27e+02   2.64e+01   5.54e+00
    3.07e+00   1.21e+00   2.39e+00   1.31e+00   1.31e+00   2.03e+00
    1.70e+00]]


In [24]:
__check_for  = 1000
silhouette_score(X[:__check_for], preds[:__check_for])

0.0032499189887050476

In [25]:
from sklearn.mixture import GaussianMixture as GMM
from sklearn.metrics import silhouette_score

# For future analysis
GMM_Centers = []

__check_for  = 100

for i in range(2, 7):
    # TODO: Apply your clustering algorithm of choice to the reduced data 
    clusterer = GMM(n_components=i, random_state=42)
    clusterer.fit(X)

    # TODO: Predict the cluster for each data point
    preds = clusterer.predict(X)

    # TODO: Find the cluster centers
    GMM_Centers.append(clusterer.means_)

    # score = silhouette_score(X, preds)
    score = silhouette_score(X[:__check_for], preds[:__check_for])

    print i, score

2 0.00900773216316
3 -0.0319390465026
4 -0.172351184764
5 -0.139534380156
6 -0.231977188575


In [27]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# For future analysis
KMM_Centers = []

# Testing each category
for i in range(2, 7):

    clusterer = KMeans(init='k-means++', n_clusters=i, n_init=10)
    clusterer.fit(X)

    preds = clusterer.predict(X)

    centers = clusterer.cluster_centers_
    
    KMM_Centers.append(centers)

#     score = silhouette_score(X, preds)
    score = silhouette_score(X[:__check_for], preds[:__check_for])
    print i, score

2 0.505801576136
3 0.334520314762
4 0.270235669827
5 0.256475256588
6 0.249260465848


In [29]:
i = 2

clusterer = KMeans(init='k-means++', n_clusters=i, n_init=10)
clusterer.fit(X)
preds = clusterer.predict(X)

In [30]:
score = silhouette_score(X[:1000], preds[:1000])
print i, score

2 0.495739502324


In [31]:
print X.shape, TEST_X.shape

(59400, 25) (14850, 25)


In [32]:
X = pd.DataFrame(X)
X['new'] = clusterer.predict(X)

In [33]:
TEST_X = pd.DataFrame(TEST_X)
TEST_X['new'] = clusterer.predict(TEST_X)

In [34]:
print X.shape, TEST_X.shape

(59400, 26) (14850, 26)


## Supervised Learning

* Supervised Learning(GBT Trees, Nearest Neighbours, RF, One-vs-One)

### Test-Train Split

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import xgboost as xgb



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# print features.shape
# X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.25, random_state=42, stratify=y)

### GBT Trees

In [ ]:
gbt_clf = GradientBoostingClassifier(n_estimators=150, max_depth=5)

In [ ]:
gbt_clf = gbt_clf.fit(X_train, y_train)

print 'score:', gbt_clf.score(X_test, y_test)

### Nearest Neighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_clf = KNeighborsClassifier()

In [ ]:
knn_clf.fit(X_test, y_test)

In [ ]:
knn_clf.score(X_train, y_train)

### Random Forest

In [36]:
rf_clf = RandomForestClassifier(n_estimators=150, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1)
# class_weight="balanced_subsample"/"balanced"
# criterion="gini"/"entropy"

rf_clf = rf_clf.fit(X_train, y_train)
# pred = rf_clf.predict_proba(X_test)
rf_clf.score(X_test, y_test)

# (n_estimators=100, class_weight="balanced_subsample", n_jobs=-1) 0.80782828282828278
# (n_estimators=100, class_weight="balanced_subsample", n_jobs=-1) 0.81186868686868685
# (n_estimators=150, class_weight="balanced_subsample", n_jobs=-1) 0.8113636363636364

# (n_estimators=150, criterion='gini', class_weight="balanced_subsample", n_jobs=-1) 0.81018518518518523
# (n_estimators=150, criterion='gini', class_weight="balanced", n_jobs=-1) 0.80858585858585863
# (n_estimators=150, criterion='gini', n_jobs=-1) 0.80942760942760938
# (n_estimators=150, criterion='entropy', n_jobs=-1) 0.81060606060606055

# (n_estimators=10, criterion='gini', class_weight="balanced_subsample", n_jobs=-1) 0.79781144781144786
# (n_estimators=10, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1) 0.80185185185185182
# (n_estimators=150, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1) 0.8113636363636364
# (n_estimators=150, criterion='gini', class_weight="balanced", n_jobs=-1) 0.80984848484848482

# (n_estimators=150, criterion='gini', class_weight="balanced", n_jobs=-1) 0.81259259259259264
# (n_estimators=150, criterion='gini', class_weight="balanced_subsample", n_jobs=-1) 0.81259259259259264
# (n_estimators=150, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1) 0.81252525252525254

# 0.81198653198653203

0.80983164983164979

### One-vs-One

In [37]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

In [ ]:
multiclass_rf_clf = OneVsOneClassifier(RandomForestClassifier(n_estimators=100,
                                                        criterion='entropy',
#                                                         class_weight="balanced_subsample",
                                                        n_jobs=-1))

# 0.81265993265993264

In [38]:
multiclass_rf_clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=100,
                                                        criterion='entropy',
                                                        class_weight="balanced_subsample",
                                                        n_jobs=-1))

# 0.81393939393939396

# 0.81272727272727274

In [39]:
multiclass_rf_clf = multiclass_rf_clf.fit(X_train, y_train)

multiclass_rf_clf.score(X_test, y_test)

0.81138047138047142

In [40]:
multiclass_rf_clf

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          n_jobs=1)

## Parameter tuning

In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [42]:
parameters = {
    'n_estimators': [10, 50, 100, 150],
    'class_weight': 'balanced_subsample balanced'.split(),
    'criterion': 'gini entropy'.split()
}
GS_CV = GridSearchCV(estimator=RandomForestClassifier(), param_grid=parameters, n_jobs=-1)
# GS_CV = RandomizedSearchCV(RandomForestClassifier(), parameters)

GS_CV.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 50, 100, 150], 'criterion': ['gini', 'entropy'], 'class_weight': ['balanced_subsample', 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [43]:
print GS_CV.best_params_, GS_CV.best_score_

{'n_estimators': 100, 'criterion': 'entropy', 'class_weight': 'balanced'} 0.80595959596


In [44]:
for each in GS_CV.grid_scores_:
    print str(each).replace(' ', '')

mean:0.79148,std:0.00165,params:{'n_estimators':10,'criterion':'gini','class_weight':'balanced_subsample'}
mean:0.80429,std:0.00242,params:{'n_estimators':50,'criterion':'gini','class_weight':'balanced_subsample'}
mean:0.80485,std:0.00225,params:{'n_estimators':100,'criterion':'gini','class_weight':'balanced_subsample'}
mean:0.80562,std:0.00227,params:{'n_estimators':150,'criterion':'gini','class_weight':'balanced_subsample'}
mean:0.79114,std:0.00215,params:{'n_estimators':10,'criterion':'entropy','class_weight':'balanced_subsample'}
mean:0.80273,std:0.00073,params:{'n_estimators':50,'criterion':'entropy','class_weight':'balanced_subsample'}
mean:0.80426,std:0.00153,params:{'n_estimators':100,'criterion':'entropy','class_weight':'balanced_subsample'}
mean:0.80507,std:0.00238,params:{'n_estimators':150,'criterion':'entropy','class_weight':'balanced_subsample'}
mean:0.79249,std:0.00140,params:{'n_estimators':10,'criterion':'gini','class_weight':'balanced'}
mean:0.80401,std:0.00248,params

/Users/sampathkumarm/anaconda/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [ ]:
sorted(clf.cv_results_.keys())

In [ ]:
len(clf.feature_importances_)

In [ ]:
plt.figure(figsize=(12, 3))

tmp = clf.feature_importances_

# making importance relative
a, b = min(tmp), max(tmp)
cols_imp = (tmp - a) /b
_ = plt.scatter(range(30), cols_imp)
_ = plt.plot((0, 29), (0.15,0.15), '-r')
_ = plt.xlabel('Columns')
_ = plt.ylabel('Relative Col Importance')

In [ ]:
print map(lambda x: len(x), [X_test, y_test])

clf.score(X_test, y_test) # 0.81097643097643102

### XGBOOST

In [48]:

# gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=100, learning_rate=0.05).fit(X_train, y_train)

gbm_predictions = gbm.predict(X_test)

160498800.0


In [49]:
print sum(gbm_predictions == y_test)/ (1.0 * len(y_test)) # 0.7279461279461279

0.727811447811


## Submission

In [ ]:
test_ids = RAW_TEST_X.index

predictions = clf.predict(TEST_X)

print (predictions.shape)

predictions_labels = le.inverse_transform(predictions)

# sub = pd.DataFrame(predictions, columns=list(le.classes_))
sub = pd.DataFrame(predictions_labels, columns=['status_group'])
sub.head()

sub.insert(loc=0, column='id', value=test_ids)
sub.reset_index()
sub.to_csv('submit.csv', index = False)
sub.head()